In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
from IPython.core.display import clear_output
import ase, itertools
from ase.visualize import view
from ase import Atoms
from copy import deepcopy
from scipy.spatial.distance import pdist
from itertools import product

# generate random cell

In [ ]:
# Hexagonal v1 = a(1,0,0),  v2 = a(-1/2,sqrt(3)/2,0),  v3 = a(0,0,c/a)
a = 2
c = 2
nat = 10    # number of atoms
rrange = 5.0 # range of x positions
dim = 3      # dimensionality
cutoff = 3.0 # cutoff for interactions
h = a*np.array([[1,0,0],[-0.5,np.sqrt(3)/2,0],[0,0,c/a]])
h = a*np.array([[1,0,0],[-2,np.sqrt(3)/2,0],[0,0,c/a]])
r = np.random.uniform(low=-rrange, high=2*rrange, size=(nat, dim))
pbc = [True,True,True]
frame = Atoms(positions=r,cell=h,numbers=nat*[1,],pbc=pbc)
# view(frame)
# frame.wrap()
# r = frame.positions

In [ ]:
aa = deepcopy(frame)
aa.wrap()
view([frame,aa])
# view(frame)

In [ ]:
cell = frame.cell
pos = frame.positions
np.dot(np.linalg.inv(cell).T,pos.T).T

In [ ]:
frame.positions

In [ ]:
frame.get_scaled_positions(wrap=False) % 1.0

In [ ]:
aa.positions

In [ ]:
view(frame)

In [ ]:
ase.io.write('../reference_data/outputs/dummy_structure.json',frame)
ase.io.write('../reference_data/outputs/dummy_structure_wrapped.json',aa)

# builds neighbor list

## determines bounding box and periodic images

bounding box, including a cutoff-sized skin. this is the range we need to cover with periodic copies of the cell to find neighbors for all atoms

In [ ]:
frame = ase.io.read('../reference_data/outputs/test.json')
r = frame.positions
nat = len(frame)
h = frame.cell
pbc = frame.pbc


In [ ]:
bb = np.asarray([ r.min(axis=0), r.max(axis=0) ])
bb[0] -= cutoff
bb[1] += cutoff
bb[0] -= cutoff
bb[1] += cutoff
bb

In [ ]:
def isotropic_growth():
    images = []
    bounds_p = np.ones(3,int)
    bounds_m = -np.ones(3,int)
    carry_on = True
    norm_bound = np.array([1,2])
    while carry_on:
        updates = False
        tests = []
        for ix in range(bounds_m[0],bounds_p[0]+1):
            for iy in range(bounds_m[1],bounds_p[1]+1):
                for iz in range(bounds_m[2],bounds_p[2]+1):
                    ic = np.array([ix,iy,iz])
                    ic_n = np.linalg.norm(ic)
                    if ic_n < norm_bound[0] or ic_n >= norm_bound[1]:
                        continue
                    tests.append(ic_n)
                    offset = np.dot(ic,h)
                    ro = r + offset
                    for ri in ro:
                        if np.all(ri>=bb[0]) and np.all(ri<=bb[1]):
                            images.append(ri)
                            updates = True
        # print(bounds_p,bounds_m)
        if updates is False:
            carry_on = False
        bounds_p += 1
        bounds_m -= 1
        norm_bound += 1

    im = np.asarray(images)     
    nreplicas = np.prod(bounds_p - bounds_m)
    return im, nreplicas

def grow_and_check(ic, pcells, images):
    for d in product([-1, 0, 1],repeat=dim):
        ncd = tuple(np.asarray(ic,int)+np.asarray(d))
        if ncd in pcells:
            continue
        offset = np.dot(np.asarray(ncd),h)
        ro = r + offset
        found_any = False
        for ri in ro:
            if all(ri>=bb[0]) and all(ri<=bb[1]):
                found_any = True
                images.append(ri)  
        pcells.append(ncd)
        if found_any:
            grow_and_check(ncd, pcells, images)
    return np.asarray(images), len(pcells)

im_iso, nreplicas_iso = isotropic_growth()
ic = np.zeros(dim, int)
pcells = [tuple(ic)]
images = []
im_it, nreplicas_it = grow_and_check(ic,pcells,images)

print(len(im_iso),nreplicas_iso,len(im_it),nreplicas_it)



In [ ]:
np.min(pcells,axis=0), np.max(pcells,axis=0)

In [ ]:
aa = np.min(pcells,axis=0)
print(aa[:100])

In [ ]:
dd = pdist(im_it)
dd[np.abs(dd) < 1e-3]

In [ ]:
im = im_it
frame_filled = Atoms(positions=np.vstack([r,im]),cell=h,numbers=nat*[8,]+len(im)*[1],pbc=pbc)
view(frame_filled)

## builds linked cells

since we computed all the necessary images, we can make linked cells that are exactly rc

In [ ]:
ncells = np.asarray(np.ceil((bb[1]-bb[0])/cutoff),int)
ncells

to which cell do each point belong?

In [ ]:
icr = np.asarray(np.floor((r-bb[0])/cutoff),int)
icim = np.asarray(np.floor((im-bb[0])/cutoff),int)
icr

In [ ]:
class Boxes(object):
    def __init__(self,ncells):
        self.ncells = ncells
        self.boxes = []
        for ii in range(np.prod(ncells)):
            self.boxes.append([])
    def get_lin_idx(self, pos_3d):
        lin_idx = 0
        fac = 1
        for ii,p in enumerate(pos_3d):
            lin_idx += fac*p
            fac *= self.ncells[ii]
        return lin_idx
    def get_mult_idx(self, lin_idx):
        pos_3d = np.array([0,0,0],int)
        for ii in range(3):
            pos_3d[ii] = lin_idx % self.ncells[ii]
            lin_idx /= self.ncells[ii]
        return pos_3d
    def __getitem__(self,pos_3d):
        return self.boxes[self.get_lin_idx(pos_3d)]
    def get_size(self,pos_3d):
        return len(self.boxes[self.get_lin_idx(pos_3d)])
    def __repr__(self):
        return str(self.boxes)
    def __str__(self):
        return self.__repr__()
    def __iter__(self):
        for box in self.boxes:
            yield box
    def get_neighbour_boxes(self,lin_idx):
        pos_3d = self.get_mult_idx(lin_idx)
        # print(pos_3d)
        
        for ix,iy,iz in product([-1, 0, 1],repeat=dim):
            if ix == 0 and iy == 0 and iz == 0: continue
            lin_idx = self.get_lin_idx(pos_3d+np.array([ix,iy,iz]))
            if len(self.boxes[lin_idx]) > 0:
                yield self.boxes[lin_idx]

In [ ]:
ci = Boxes(ncells)
for i, ic in enumerate(icr):    
    ci[ic].append(i)

In [ ]:
for ix in range(4):
    for iy in range(4):
        for iz in range(3):
            aa = [ix,iy,iz]
            lin_idx = ci.get_lin_idx(aa)
            out = ci.get_mult_idx(lin_idx)
            print(aa,out)


In [ ]:
ciim = Boxes(ncells)
for i, ic in enumerate(icim):    
    ciim[ic].append(i+nat)

just put all atoms in the neighboring cells as neighbors

In [ ]:
# array with all positions
all_pos = np.vstack([r,im])
# the NL
NL = [[] for ii in range(nat)]
for ibox_lin, cbox in enumerate(ci):
    cbox_np = np.array(cbox)
    for center_idx in cbox:
        # print(center_idx)
        # add centers that are in the same box
        NL[center_idx].extend(list(cbox_np[cbox_np != center_idx]))
        # add centers from the neighboring boxes
        for neigh_box in ci.get_neighbour_boxes(ibox_lin):
            NL[center_idx].extend(neigh_box)
        # add images from the same box
        NL[center_idx].extend(ciim.boxes[ibox_lin])
        # add images from the neighboring boxes
        for neigh_box in ciim.get_neighbour_boxes(ibox_lin):
            NL[center_idx].extend(neigh_box)
distances = [[] for ii in range(nat)]
for icenter in range(nat):
    for ineigh in NL[icenter]:
        dist = np.linalg.norm(all_pos[icenter] - all_pos[ineigh])
        distances[icenter].append(dist)

In [ ]:
from ase.io import read
ff = read('../reference_data/inputs/alloy-small.json')
cell = ff.cell
positions = ff.positions

In [ ]:
fractional = np.linalg.solve(cell.T,
                                 np.asarray(positions).T).T

In [ ]:
aa = ase.Atoms(positions=[[0,0,0],[ 0.5, 0.288675, 0.816497]],cell=[[ 1   ,     0  ,      0],[-0.5, 0.866025  ,      0],[ 0   ,     0,  1.63299]])
view(aa)

In [ ]:
aa = ase.Atoms(positions=[[0,0,0],[ 0.,  0.816497 , 0.57735     ]],cell=[[ 1   ,     0  ,      0],[-0.,  1.63299  ,      0],[ 0.5  ,     0,  0.866025]])
view(aa)

In [ ]:
np.diag([1.,1.,1/4.]) * aa.cell

In [ ]:
-1.5 % 1.

In [ ]:
aa = np.linalg.inv(cell)
frac = np.dot(positions, aa.T)

In [ ]:
fractional-frac

In [ ]:
icenter = 2

In [ ]:
dd1 = np.array(distances[icenter])
ee = np.sort(dd1[dd1 <= cutoff])
ee

In [ ]:
dd1 = np.array(distances[icenter])
dd = np.sort(dd1[dd1 <= cutoff])
dd

In [ ]:
ee - dd

# The whole neighbourlist design streamlined

In [ ]:
# grow the cell shell by shell in an all directions. stops when no replicas of a shell add atom to the bounding box
def isotropic_growth(h,bb):
    images = []
    bounds_p = np.ones(3,int)
    bounds_m = -np.ones(3,int)
    carry_on = True
    norm_bound = np.array([1,2])
    while carry_on:
        updates = False
        tests = []
        for ix in range(bounds_m[0],bounds_p[0]+1):
            for iy in range(bounds_m[1],bounds_p[1]+1):
                for iz in range(bounds_m[2],bounds_p[2]+1):
                    ic = np.array([ix,iy,iz])
                    ic_n = np.linalg.norm(ic)
                    if ic_n < norm_bound[0] or ic_n >= norm_bound[1]:
                        continue
                    tests.append(ic_n)
                    offset = np.dot(ic,h)
                    ro = r + offset
                    for ri in ro:
                        if np.all(ri>=bb[0]) and np.all(ri<=bb[1]):
                            images.append(ri)
                            updates = True
        # print(bounds_p,bounds_m)
        if updates is False:
            carry_on = False
        bounds_p += 1
        bounds_m -= 1
        norm_bound += 1

    im = np.asarray(images)     
    nreplicas = np.prod(bounds_p - bounds_m)
    return im, nreplicas

# grow the cell recursivelly: 
# add neighboring replicas to pcells
# go through the replicas in pcells, if atoms are added to the bounding box then add the neighgoring cells to pcells 
# making sure pcells has unique entries
def grow_and_check(r,h,bb,pbc):
    dim = len(pbc)
    images = []
    pcells = [tuple(np.zeros(dim))]
    aa = []
    for ii in range(dim):
        if pbc[ii] == True:
            aa.append([-1,0,1])
        else:
            aa.append([0])
    
    def func_recursive(ic):
        for d in itertools.product(*aa):
            ncd = tuple(np.asarray(ic,int)+np.asarray(d))
            if ncd in pcells:
                continue
            offset = np.dot(np.asarray(ncd),h)
            ro = r + offset
            found_any = False
            for ri in ro:
                if np.all(ri>=bb[0]) and np.all(ri<=bb[1]):
                    found_any = True
                    images.append(ri)  
            pcells.append(ncd)
            if found_any is True:
                func_recursive(ncd)
    func_recursive(pcells[0])
    return np.asarray(images), len(pcells)

In [ ]:
aa

In [ ]:
pbc = [True,False,True]
aa = []
for ii in range(len(pbc)):
    if pbc[ii] == True:
        aa.append([-1,0,1])
    else:
        aa.append([0])
list(itertools.product(*aa))

In [ ]:
# representation of the binning boxes within the bounding box
class Boxes(object):
    def __init__(self,ncells,pbc):
        self.pbc = pbc
        self.ncells = ncells
        self.boxes = []
        for ii in range(np.prod(ncells)):
            self.boxes.append([])
    def get_lin_idx(self, pos_3d):
        lin_idx = 0
        fac = 1
        for ii,p in enumerate(pos_3d):
            lin_idx += fac*p
            fac *= self.ncells[ii]
        return lin_idx
    def get_mult_idx(self, lin_idx):
        pos_3d = np.array([0,0,0],int)
        for ii in range(3):
            pos_3d[ii] = lin_idx % self.ncells[ii]
            lin_idx /= self.ncells[ii]
        return pos_3d
    def __getitem__(self,pos_3d):
        return self.boxes[self.get_lin_idx(pos_3d)]
    def __iter__(self):
        for box in self.boxes:
            yield box
    def get_neighbour_boxes(self,lin_idx):
        pos_3d = self.get_mult_idx(lin_idx)
        for ix,iy,iz in product([-1, 0, 1],repeat=dim):
            # avoid centeral box 
            if ix == 0 and iy == 0 and iz == 0: 
                continue
            lin_idx = self.get_lin_idx(pos_3d+np.array([ix,iy,iz]))
            if len(self.boxes[lin_idx]) > 0:
                yield self.boxes[lin_idx]
    def __repr__(self):
        return str(self.boxes)
    def __str__(self):
        return self.__repr__()
    def get_size(self,pos_3d):
        return len(self.boxes[self.get_lin_idx(pos_3d)])

In [ ]:
def get_neighborlist(frame, cutoff):
    nat = len(frame)
    h = frame.get_cell()
    r = frame.get_positions()
    pbc = frame.get_pbc()
    # bounding box, including a cutoff-sized skin. this is the range we need to cover 
    # with periodic copies of the cell to find neighbors for all atoms
    bb = np.asarray([ r.min(axis=0), r.max(axis=0) ])
    bb[0] -= cutoff
    bb[1] += cutoff
    # build the images iterativelly by growing the replication of the cell from (0,0,0)
    im, nnn = grow_and_check(r, h, bb, pbc)

    # since we computed all the necessary images, we can make linked cells that are exactly rc
    ncells = np.asarray(np.ceil((bb[1]-bb[0])/cutoff),int)
    # to which cell do each point belong?
    icr = np.asarray(np.floor((r-bb[0])/cutoff),int)
    icim = np.asarray(np.floor((im-bb[0])/cutoff),int)

    # bin the centers in the boxes
    ci = Boxes(ncells,pbc)
    for i, ic in enumerate(icr):    
        ci[ic].append(i)
    # bin the replicas
    ciim = Boxes(ncells,pbc)
    for i, ic in enumerate(icim):    
        ciim[ic].append(i+nat)    

    # array with all positions
    all_pos = np.vstack([r,im])

    # build the NL
    NL = [[] for ii in range(nat)]
    for ibox_lin, cbox in enumerate(ci):
        cbox_np = np.array(cbox)
        for center_idx in cbox:
            # add centers that are in the same box
            NL[center_idx].extend(list(cbox_np[cbox_np != center_idx]))
            # add centers from the neighboring boxes
            for neigh_box in ci.get_neighbour_boxes(ibox_lin):
                NL[center_idx].extend(neigh_box)
            # add images from the same boxe
            NL[center_idx].extend(ciim.boxes[ibox_lin])
            # add images from the neighboring boxes
            for neigh_box in ciim.get_neighbour_boxes(ibox_lin):
                NL[center_idx].extend(neigh_box)

    distances = [[] for ii in range(nat)]
    for icenter in range(nat):
        for ineigh in NL[icenter]:
            dist = np.linalg.norm(all_pos[icenter] - all_pos[ineigh])
            distances[icenter].append(dist)

    return NL, distances, all_pos

In [ ]:
# Hexagonal v1 = a(1,0,0),  v2 = a(-1/2,sqrt(3)/2,0),  v3 = a(0,0,c/a)
a = 3
c = 2
nat = 10    # number of atoms
rrange = 5.0 # range of x positions
dim = 3      # dimensionality
cutoff = 3.0 # cutoff for interactions
# h = a*np.array([[1,0,0],[-0.5,np.sqrt(3)/2,0],[0,0,c/a]])
h = a*np.array([[1,0,0],[-2,np.sqrt(3)/2,0],[0,0,c/a]])
r = np.random.uniform(low=-rrange, high=2*rrange, size=(nat, dim))
pbc = [True,False,True]
frame = Atoms(positions=r,cell=h,numbers=nat*[1,],pbc=pbc)
frame_wraped = deepcopy(frame)
frame_wraped.wrap()

In [ ]:
view([frame,frame_wraped])

In [ ]:
NL_1, distances_1, all_pos_1 = get_neighborlist(frame, cutoff)
NL_2, distances_2, all_pos_2 = get_neighborlist(frame_wraped, cutoff)

In [ ]:
epsilon = 1e-14
# check if the neighbour distances to the centers in both NL are the same in a strict cutoff sense
for icenter in range(nat):
    # icenter = 3
    dd1 = np.array(distances_1[icenter])
    dd2 = np.array(distances_2[icenter])
    ee = np.sort(dd1[dd1 <= cutoff+epsilon])
    dd = np.sort(dd2[dd2 <= cutoff+epsilon])
    print(icenter, np.allclose(ee,dd))